In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [7]:
df['time'].unique()

array([0, 1])

In [8]:
X = df.drop('time', axis=1)
y = df['time']

In [9]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [10]:
y

,time
0,0
1,0
2,0
3,0
4,0
...,...
239,0
240,0
241,0
242,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
X_train.shape, X_test.shape

((195, 6), (49, 6))

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
cat_cols = ["sex", "smoker", "day"]
num_cols = ["total_bill", "tip", "size"]

In [15]:
num_pipeline = Pipeline( steps = [('imputation', SimpleImputer(strategy='median')),
                                  ('scaling', StandardScaler())])

cat_pipeline = Pipeline( steps = [('imputation', SimpleImputer(strategy='most_frequent')),
                                  ('encoding', OneHotEncoder())])

In [16]:
ColumnTransformer([('num_pipeline', num_pipeline, num_cols),
                   ('cat_pipeline', cat_pipeline, cat_cols)])

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [17]:
preprocessor = ColumnTransformer([('num_pipeline', num_pipeline, num_cols),
                                  ('cat_pipeline', cat_pipeline, cat_cols)])

In [18]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [20]:
models = {'DT': DecisionTreeClassifier(),
          'SVM': SVC(),
          'logistic': LogisticRegression()}

In [21]:
for i in range(len(models)):
    print(list(models.values())[i])

DecisionTreeClassifier()
SVC()
LogisticRegression()


In [22]:
from sklearn.metrics import accuracy_score

In [23]:
def model_train_eval(X_train, X_test, y_train, y_test, models):
  evaluation = {}
  for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_score = accuracy_score(y_test, y_pred)
    evaluation[list(models.keys())[i]] = model_score
  return evaluation

In [24]:
model_train_eval(X_train, X_test, y_train, y_test, models)

{'DT': 0.8979591836734694,
 'SVM': 0.9183673469387755,
 'logistic': 0.9183673469387755}